In [2]:
import numpy as np
import pandas as pd
import sys
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
from generating_feature import generating_pro_feature, generating_drug_feature, concat_feature
sys.path = original_path

/home/max38744/.conda/envs/moon/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
#pd.read_csv("convert_table", delimiter="\t")
df

,0,1
0,Q16348,DB00672
1,P13995,DB00116
2,P30872,DB06663
3,P30874,DB06663
4,O43526,DB00586
...,...,...
9876,O60488,DB00412
9877,P51843,DB00755
9878,Q16348,DB01140
9879,P50416,DB01016


In [7]:
file_path = "R_DrugBank.txt"  # Specify the file path without the extension

# Read the file as a CSV with custom parameters

df = pd.read_csv(file_path, header=None, delimiter= "\t")  # Assuming the file has no header row
print(len(df))
print(df.iloc[[0,1,300,600, 1000]])



9881
           0        1
0     Q16348  DB00672
1     P13995  DB00116
300   Q16739  DB00419
600   P21917  DB00714
1000  P41145  DB00497


In [72]:
ids = []
for i in df[0] :
    ids.append(i)
np.savetxt("pro_list.txt", np.array(ids), fmt="%s")

In [71]:
from pyfaidx import Fasta
sequences = Fasta('uniprot_sprot.fasta')

ModuleNotFoundError: No module named 'pyfaidx'

In [47]:
import requests

# Set the base URL for the UniProt API
base_url = "https://www.uniprot.org/uniprot"

def get_seq(uni_id) :
    # Set the UniProtID of the protein you want to query
    uniprot_id = uni_id

    # Set the headers with the desired response format
    headers = {"Accept": "text/plain"}

    # Make a GET request to the UniProt API to retrieve the protein sequence
    response = requests.get(f"{base_url}/{uniprot_id}.fasta", headers=headers)


    # Check if the request was successful
    if response.status_code == 200:
        # Get the protein sequence from the response
        seq = ""
        protein_sequence = "".join(response.text.split("\n")[1:])

    else:
        # Print an error message
        protein_sequence = "None"
        
    return uni_id, protein_sequence

pro_id_list = []
seq_list = []

for i in df[1].values[:4000] :
    pro, seq = get_seq(i)
    pro_id_list.append(pro)
    seq_list.append(seq)


In [31]:
import requests

# Set the base URL for the DrugBank API
base_url = "https://api.drugbank.com/v2"

# Set the DrugBankID of the drug you want to query
drugbank_id = "DB00001"

# Set your DrugBank API key
api_key = "YOUR_API_KEY_HERE"

# Set the headers with the API key
headers = {"Authorization": f"Bearer {api_key}"}

# Make a GET request to the DrugBank API to retrieve the drug information
response = requests.get(f"{base_url}/us/drugs/{drugbank_id}", headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Get the drug information from the response
    drug_info = response.json()

    # Get the SMILES value from the drug information
    smiles = drug_info["smiles"]

    # Print the SMILES value
    print(f"SMILES: {smiles}")
else:
    # Print an error message
    print(f"An error occurred: {response.text}")


An error occurred: {"error":"Token invalid"}


In [ ]:
# Select unique rows based on the 'Drug' column
unique_drugs_df = df.drop_duplicates(subset=['Drug'])

unique_drugs_df = generating_drug_feature(unique_drugs_df)
# Print the resulting DataFrame
# print(len(unique_drugs_df))
# print(unique_drugs_df)
tmp = unique_drugs_df[['Drug_ID', 'Drug','Morgan_Features']]
# print(tmp.head())

drug_id_array = tmp['Drug_ID'].values
drug_array = tmp['Drug'].values
drug_feat = tmp['Morgan_Features']


drug_feat = {'Drug_ID': drug_id_array, 'Drug': drug_array, 'Morgan_Features': drug_feat}

print(len(drug_feat['Morgan_Features']))
print(type(drug_feat['Morgan_Features']))


import pickle


with open("DAVIS_drug_feats.pkl", 'wb') as f:
    pickle.dump(drug_feat, f)

print(f"Dictionary saved as pickle file")